## IMPORT DEPENDENCIES

In [11]:
import pandas as pd
import os
import requests
import json
from config import dbuser, dbpassword, dbhost, dbport, dbname
from sqlalchemy import create_engine

### CREATING A DATASET

In [6]:
# =============================================================== #
# # # CREATING A HEROKU DATASET
# =============================================================== #
# Heroku only takes data with <10k rows so I created a heroku dataset (incase we ever want to host on heroku).

def import_func(country_code):
    
    # Creating Path
    path = os.path.join('data','newData',f'{country_code}_youtube_trending_data.csv')
    
    # Storing dataframe to df
    dfh=pd.read_csv(path)
    
    # Removing unwanted columns below
    dfh=dfh[['video_id','title','publishedAt','channelTitle','categoryId','trending_date','view_count','likes','dislikes','comment_count','thumbnail_link']]
    
    # Renaming columns
    dfh = dfh.rename(columns={'view_count': 'views', 'likes': 'likes', 'dislikes': 'dislikes', 'comment_count': 'comments'})
    
    # Changing object types to date types for two columns
    dfh['publishedAt']=pd.to_datetime(dfh['publishedAt'])
    dfh['trending_date']=pd.to_datetime(dfh['trending_date'])
    
    # Removing time stamp from date
    dfh['publishedAt']=dfh['publishedAt'].dt.date 
    dfh['trending_date']=dfh['trending_date'].dt.date
    
    # For loop for each csv file
    with open(f'data/newData/{country_code}_category_id.json', 'r') as read_file:
        category_ids = json.load(read_file)

        dfh=dfh.astype({'categoryId': 'str'})
        
    for index,row in dfh.iterrows():
    
        for entry in category_ids["items"]:

            if row["categoryId"]==entry["id"]:
                dfh.at[index,"categoryId"]=entry["snippet"]["title"]
    
    # Select 999 rows from each country in dataframe (so we end up with <10k rows ~1000 rows per country * 10 countries)
    # dfh = dfh.nlargest(999, 'views')
    
    # Adding country Code as column
    dfh['country']=f'{country_code}'
    col_name='country'
    
    # Moving country code to first column
    first_col = dfh.pop(col_name)
    dfh.insert(0,col_name,first_col)
    
    return dfh

# USA Dataframe
dfh_us=import_func('US')

# Brasil Dataframe
dfh_br=import_func('BR')

# Canada Dataframe
dfh_ca=import_func('CA')

# Mexico Dataframe
dfh_mx=import_func('MX')

# GB Dataframe
dfh_gb=import_func('GB')

# France Dataframe
dfh_fr=import_func('FR')

# Russia Dataframe
dfh_ru=import_func('RU')

# Japan Dataframe
dfh_jp=import_func('JP')

# Korea Dataframe
dfh_kr=import_func('KR')

# India Dataframe
dfh_in=import_func('IN')


# Creating a varible to add all dfs
country_dfh=[dfh_us, dfh_br, dfh_ca, dfh_mx, dfh_gb, dfh_fr, dfh_ru ,dfh_jp, dfh_kr ,dfh_in]

# Merge output into one table
dfh_main = pd.concat(country_dfh)
dfh_main

# Fix the 29 vs Non profits issue
dfh_main['categoryId'] = dfh_main['categoryId'].replace(["29"],"Nonprofits & Activism")

# View output
dfh_main.columns




Index(['country', 'video_id', 'title', 'publishedAt', 'channelTitle',
       'categoryId', 'trending_date', 'views', 'likes', 'dislikes', 'comments',
       'thumbnail_link'],
      dtype='object')

In [10]:
# View output
dfh_main
dfh_main.to_csv(r'C:\Users\mohan\Downloads\CsvDownloadtest.csv')

### CONNECTING AND LOADING DATA INTO THE DATABASE (POSTGRES & MSSQL DBs) FOR HEROKU

In [ ]:
# =============================================================== #
# # UPLOAD DATASET TO DATABASE (POSTGRESS SQL DB)
# =============================================================== #
# You first have to go create the db and then use the same name as the db you have created
# (youtube_table_v1 < 10,000    youtube_table_v2 > 10,000) in postgres

# =============================================================== #
# # 1 Connect to Database (with Local db and all)
# =============================================================== #
pg_user = 'postgres'
pg_password = 'Sm6Jc5bqbiNQdsVAo7eN'
db_name = 'YTP_database'
connection_string = f'{pg_user}:{pg_password}@localhost:5432/{db_name}'
engine=create_engine(f'postgresql://{connection_string}')

# =============================================================== #
# 2 Connect to Database (Alternative with AWS db and all)
# =============================================================== #
# dbuser = 'postgres'
# dbpassword = 'Sm6Jc5bqbiNQdsVAo7eN'
# dbhost = 'localhost'
# dbport = '5432'
# dbname= 'YTP_database'
# connection_string2 = f'{dbuser}:{dbpassword}@database-1.cvmfiiilpm7y.us-east-1.rds.amazonaws.com:{dbport}/{dbname}'
# engine=create_engine(f'postgresql://{connection_string2}')

# =============================================================== #
# 3 Upload to postgres
# =============================================================== #
dfh_main.to_sql(name='youtube_table_v2',con=engine,if_exists='append',index=False)

In [ ]:
# =============================================================== #
# # UPLOAD DATASET TO DATABASE (SQL SERVER DB)
# =============================================================== #
# =============================================================== #
# IMPORTING DEPENDENCIES
# =============================================================== #
# from sqlalchemy import create_engine
# import urllib

# =============================================================== #
# Set up variables
# =============================================================== #
# msql_serverName = 'WORKHORSE_PC\SQLEXPRESS'
# msql_dbName = 'YTP_Database'

# =============================================================== #
# Set up connection string (This connection string is for SQL 
# server which are set up without username & pw)
# =============================================================== #

# conn_str = (
#     r'Driver=ODBC Driver 17 for SQL Server;'
#     rf'Server={msql_serverName};'
#     rf'Database={msql_dbName};'
#     r'Trusted_Connection=yes;'
# )
# quoted_conn_str = urllib.parse.quote_plus(conn_str)
# engine = create_engine(f'mssql+pyodbc:///?odbc_connect={quoted_conn_str}')
# cnxn = engine.connect()

# =============================================================== #
# Connect to DB and delete (or create) a table
# =============================================================== #

# q = """DROP TABLE youtube_table_v2"""
# cnxn.execute(q)   

# q = """ 
# CREATE TABLE youtube_table_v2 (
# country Varchar (128) NOT NULL,
# video_id Varchar (128) NOT NULL,
# title Varchar (128) NOT NULL,
# publishedAt datetime NOT NULL,
# channelTitle Varchar (128),
# categoryId Varchar (128) NOT NULL,
# trending_date datetime NOT NULL,
# views Integer NOT NULL,
# likes Integer NOT NULL,
# dislikes Integer NOT NULL,
# comments Integer NOT NULL,
# thumbnail_link Varchar (128) NOT NULL,
# )
# """

# cnxn.execute(q)  

# =============================================================== #
# Connect to DB and upload df to table
# =============================================================== #

# cnxn = engine.connect()
# dfh_main.to_sql(name = 'youtube_table_v2', con = cnxn, if_exists = 'append',index = False)
# cnxn.close()